In [40]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u

In [41]:
sta1_RA_WORLD = 174.77413750 - 9*10**(-5) #HD 101274
sta1_DEC_WORLD = -37.75375000 + 6.4*10**(-5)

sta2_RA_WORLD = 174.82283333 - 0.15/3600. #Tycho-2 7740-22-1
sta2_DEC_WORLD = -37.76239167

sta3_RA_WORLD = 174.79553333 #174.79553250 #Tycho-2 7740-529-1
sta3_DEC_WORLD = -37.83470278 #-37.83470333

k_g = 0.16
k_r = 0.08
k_i = 0.02
k_z = 0.01

#mag_refsta1_g = 13.291
#mag_refsta1_r = 12.843
#mag_refsta1_i = 12.668
#mag_refsta1_z = 12.593

mag_refsta2_g = 10.166
mag_refsta2_r = 10.229
mag_refsta2_i = 10.347
mag_refsta2_z = 10.456

mag_refsta1_g = 11.689
mag_refsta1_r = 11.491
mag_refsta1_i = 11.465
mag_refsta1_z = 11.500

#refsta1_RA = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).ra.degree #P525_E_21_10_17
#refsta1_DEC = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).dec.degree 

#refsta2_RA = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).ra.degree #P525_E_21_10_17
#refsta2_DEC = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).dec.degree 

refsta1_RA = SkyCoord('00:55:10 00:43:13', unit=(u.hourangle, u.deg)).ra.degree #PG1323_086_10_07_17
refsta1_DEC = SkyCoord('00:55:10 00:43:13', unit=(u.hourangle, u.deg)).dec.degree 

refsta2_RA = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).ra.degree #SA100_241
refsta2_DEC = SkyCoord('08:52:34 -00:39:49', unit=(u.hourangle, u.deg)).dec.degree 

In [42]:
QUASAR1_g = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/g',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/g')]

QUASAR2_g = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/g',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/g')]

refsta1_g = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g')]

refsta2_g = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g')]

In [43]:
QUASAR1_r = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/r',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/r')]

QUASAR2_r = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/r',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/r')]

refsta1_r = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r')]

refsta2_r = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r')]

In [44]:
QUASAR1_i = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/i',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/i')]

QUASAR2_i = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/i',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/i')]

refsta1_i = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i')]

refsta2_i = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i')]

In [45]:
QUASAR1_z = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/z',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/z')]

QUASAR2_z = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/z',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/z')]

refsta1_z = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z')]

refsta2_z = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z')]

In [46]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [47]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [48]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [49]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    try:
        mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
        AGNdata[mask] = float(0)
        mask = AGNdata == 0.
        rows = np.flatnonzero((~mask).sum(axis=1))
        cols = np.flatnonzero((~mask).sum(axis=0))
        AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
        #print apparature/float(pixspa)
    except:
        AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    return AGNdata/float(exptime), x1, y1


In [50]:
def comp1(qua,sta):
    '''Code to determine the relation between the AGN and the Stellar flux as determined by either me \
    or by the SExtractor program respectively'''
    comp = qua/sta
    return comp

In [51]:
def SNR(data1,data2,header,ap1,ap2):
    npix1 = np.pi*ap1**2/((header['PIXSCALE']/3600.)**2)
    npix2 = np.pi*ap2**2/((header['PIXSCALE']/3600.)**2)
    #data2_1 = ((data2-data1)/npix2)*npix1
    data1_2 = data1/10.
    data2_2 = data2/10.
    return SNRas(10.,data1_2,abs(data2_2),0,header['RON'],npix1,gain=header['GAIN'])

In [52]:
def SXtable(quas,aperture):
    header = pyfits.getheader(quas)
    phot_app = aperture*3600/header['PIXSCALE']
    np.set_printoptions(threshold=np.inf)
    sew = sewpy.SEW(params=['XMODEL_WORLD', 'YMODEL_WORLD', 'X2MODEL_WORLD', 'Y2MODEL_WORLD', 
                            'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_RADIUS(1)', 'FLUX_APER','ELLIPTICITY'],
                    config={"DETECT_THRESH":5, 'DETECT_MINAREA':20, 'DETECT_MAXAREA':30000, 
                            "PHOT_FLUXFRAC":"0.5", 'PHOT_APERTURES':2*phot_app, 
                            'PIXEL_SCALE':header['PIXSCALE']}
                    , sexpath='/usr/bin/sextractor')
    out = sew(quas)
    return out

In [53]:
def flux(quas,aperture,main_RA_WORLD,main_DEC_WORLD):
    header = pyfits.getheader(quas)
    RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[0]
    DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[1]
    output = float('nan')
    if 15 < RAmain < header['NAXIS1'] - 15 and 15 < DECmain < header['NAXIS2'] - 15: 
        out = SXtable(quas,aperture)
        #print out['table']['XMODEL_IMAGE']
        #print RAmain
        RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[0]
        DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[1]
        #print out['table']['XMODEL_IMAGE'], out['table']['YMODEL_IMAGE']
        #print RAmain, DECmain

        for j in range(len(out['table']['XMODEL_IMAGE'])):
            #output['time'][i] = header['MJD-OBS']
            if RAmain - 20. < out['table']['XMODEL_IMAGE'][j] <  RAmain + 20. and \
            DECmain - 20. < out['table']['YMODEL_IMAGE'][j] < DECmain + 20.:
                #print 'YES'
                output = out['table']['FLUX_APER'][j]
                break
    else:
        output = float('nan')
        #print 'Done'
    return output


In [54]:
def findmag(start,compare1,RA_star,DEC_star,RA_com,DEC_com,mag_com,AIR_EXTINCTION):
    aperture1 = 0.001
    #201.5441,-8.8601
    com_flu = []
    com_AIRMASS = []
    com_EXPTIME = []
    star_flu = []
    star_AIRMASS = []
    star_EXPTIME = []
    for i in range(len(compare1)):
        print 'Frame nr. ', i, compare1[i]
        header = pyfits.getheader(compare1[i])
        a1 = flux(compare1[i],2*aperture1,RA_com,DEC_com)/header['EXPTIME']
        if a1 > 0:
            #print a1
            com_flu.append(a1), com_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), com_EXPTIME.append(header['EXPTIME'])
        else:
            print a1
            com_flu.append(float('nan')), com_AIRMASS.append(float('nan')), com_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    for i in range(len(start)):
        print 'Frame nr. ', i, start[i]
        header = pyfits.getheader(start[i])
        a1 = flux(start[i],2*aperture1,RA_star,DEC_star)/header['EXPTIME']
        if a1 > 0:
            #print a1
            star_flu.append(a1), star_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), star_EXPTIME.append(header['EXPTIME'])
        else:
            star_flu.append(float('nan')), star_AIRMASS.append(float('nan')), star_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    com_flu = np.array(com_flu)
    com_AIRMASS = np.array(com_AIRMASS)
    com_EXPTIME = np.array(com_EXPTIME)
    star_flu = np.array(star_flu)
    star_AIRMASS = np.array(star_AIRMASS)
    star_EXPTIME = np.array(star_EXPTIME)
    com_flu = com_flu/com_EXPTIME
    star_flu = star_flu/star_EXPTIME
    com_flux = np.nanmean(com_flu)
    com_AIRMASS_num = np.nanmean(com_AIRMASS)
    star_AIRMASS_num = np.nanmean(star_AIRMASS)
    mag = -(star_AIRMASS_num-com_AIRMASS_num) + mag_com - 2.5*np.log10(star_flu/float(com_flux))
    print mag
    return mag

In [55]:
com1_1 = findmag(QUASAR1_g,refsta1_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com2_1 = findmag(QUASAR1_g,refsta1_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com3_1 = findmag(QUASAR1_g,refsta1_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)

com1_2 = findmag(QUASAR2_g,refsta2_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)
com2_2 = findmag(QUASAR2_g,refsta2_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)
com3_2 = findmag(QUASAR2_g,refsta2_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)

Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g/IMG2017315URh001.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g/IMG2017315URh002.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g/IMG2017315URh005.fits
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g/IMG2017315URh006.fits
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g/IMG2017315URs003.fits
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g/IMG2017315URs004.fits
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g/IMG2017315URs007.fits
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/g/IMG2017315URs008.fits
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/g/IMG2017315URs129.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/g/IMG2017315URs130.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/13

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice


[ nan  nan  nan]
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g/IMG2016158URh009.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g/IMG2016158URh010.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g/IMG2016158URh013.fits
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g/IMG2016158URh014.fits
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g/IMG2016158URs011.fits
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g/IMG2016158URs012.fits
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g/IMG2016158URs015.fits
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/g/IMG2016158URs016.fits
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/g/IMG2016158URs046.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/g/IMG2016158URs047.fits
Frame nr.  2 /media/lynge/Lynge-Back-

In [56]:
print 'g-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

g-filter
Star 1
[ 10.13540332  10.14115504  10.11864753] [ 10.18804222  10.23108917  10.19711111]
10.1685747334 0.0396784374978
Star 2
[ 13.20204375  13.18690662  13.17314709] [ 13.35916293  13.34037592  13.33508983]
13.2661210229 0.0795322297974
Star 3
[ nan  nan  nan] [ nan  nan  nan]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [57]:
com1_1 = findmag(QUASAR1_r,refsta1_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com2_1 = findmag(QUASAR1_r,refsta1_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com3_1 = findmag(QUASAR1_r,refsta1_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)

com1_2 = findmag(QUASAR2_r,refsta2_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)
com2_2 = findmag(QUASAR2_r,refsta2_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)
com3_2 = findmag(QUASAR2_r,refsta2_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)

Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r/IMG2017315BRh001.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r/IMG2017315BRh002.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r/IMG2017315BRh005.fits
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r/IMG2017315BRh006.fits
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r/IMG2017315BRs003.fits
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r/IMG2017315BRs004.fits
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r/IMG2017315BRs007.fits
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/r/IMG2017315BRs008.fits
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/r/IMG2017315BRs129.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/r/IMG2017315BRs130.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/13

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice


[ nan  nan  nan]
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r/IMG2016158BRh009.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r/IMG2016158BRh010.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r/IMG2016158BRh013.fits
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r/IMG2016158BRh014.fits
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r/IMG2016158BRs011.fits
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r/IMG2016158BRs012.fits
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r/IMG2016158BRs015.fits
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/r/IMG2016158BRs016.fits
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/r/IMG2016158BRs046.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/r/IMG2016158BRs047.fits
Frame nr.  2 /media/lynge/Lynge-Back-

In [58]:
print 'r-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

r-filter
Star 1
[ 10.27621446  10.28612562  10.26646825] [ 10.30261981  10.34778423  10.3178402 ]
10.2995087619 0.0273546140047
Star 2
[ 12.75972246  12.75813154  12.75973464] [ 12.80865082  12.80182579  12.79904004]
12.7811842147 0.0221789549241
Star 3
[ nan  nan  nan] [ nan  nan  nan]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [59]:
com1_1 = findmag(QUASAR1_i,refsta1_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com2_1 = findmag(QUASAR1_i,refsta1_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com3_1 = findmag(QUASAR1_i,refsta1_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)

com1_2 = findmag(QUASAR2_i,refsta2_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)
com2_2 = findmag(QUASAR2_i,refsta2_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)
com3_2 = findmag(QUASAR2_i,refsta2_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)

Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i/IMG2017315ULh001.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i/IMG2017315ULh002.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i/IMG2017315ULh005.fits
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i/IMG2017315ULh006.fits
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i/IMG2017315ULs003.fits
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i/IMG2017315ULs004.fits
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i/IMG2017315ULs007.fits
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/i/IMG2017315ULs008.fits
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/i/IMG2017315ULs129.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/i/IMG2017315ULs130.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/13

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice


[ nan  nan  nan]
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i/IMG2016158ULh009.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i/IMG2016158ULh010.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i/IMG2016158ULh013.fits
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i/IMG2016158ULh014.fits
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i/IMG2016158ULs011.fits
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i/IMG2016158ULs012.fits
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i/IMG2016158ULs015.fits
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/i/IMG2016158ULs016.fits
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/i/IMG2016158ULs046.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/i/IMG2016158ULs047.fits
Frame nr.  2 /media/lynge/Lynge-Back-

In [60]:
print 'i-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

i-filter
Star 1
[ 10.37454967  10.38998563  10.36147763] [ 10.40675171  10.39160153  10.39330886]
10.3862791713 0.0145103670475
Star 2
[ 12.67871402  12.67742182  12.67517737] [ 12.75149991  12.73956632  12.74061702]
12.7104994102 0.0336280691626
Star 3
[ nan  nan  nan] [ nan  nan  nan]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [61]:
com1_1 = findmag(QUASAR1_z,refsta1_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com2_1 = findmag(QUASAR1_z,refsta1_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com3_1 = findmag(QUASAR1_z,refsta1_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)

com1_2 = findmag(QUASAR2_z,refsta2_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)
com2_2 = findmag(QUASAR2_z,refsta2_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)
com3_2 = findmag(QUASAR2_z,refsta2_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)

Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z/IMG2017315BLh001.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z/IMG2017315BLh002.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z/IMG2017315BLh005.fits
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z/IMG2017315BLh006.fits
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z/IMG2017315BLs003.fits
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z/IMG2017315BLs004.fits
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z/IMG2017315BLs007.fits
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/13-11-17-SA092_342/z/IMG2017315BLs008.fits
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/z/IMG2017315BLs129.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/13-11-17-NGC3783/z/IMG2017315BLs130.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/13

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice


[ nan  nan  nan]
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z/IMG2016158BLh009.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z/IMG2016158BLh010.fits
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z/IMG2016158BLh013.fits
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z/IMG2016158BLh014.fits
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z/IMG2016158BLs011.fits
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z/IMG2016158BLs012.fits
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z/IMG2016158BLs015.fits
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/08-06-16-SA100-241/z/IMG2016158BLs016.fits
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/z/IMG2016158BLs046.fits
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/08-06-16-NGC3783/z/IMG2016158BLs047.fits
Frame nr.  2 /media/lynge/Lynge-Back-

In [62]:
print 'z-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

z-filter
Star 1
[ 10.48740034  10.47747231  10.47658221] [ 10.58614041  10.57174144  10.57640555]
10.5292903776 0.0491119411049
Star 2
[ 12.87244799  12.86246751  12.85006374] [ 13.25204919  13.2091248   13.18694293]
13.0388493596 0.178334694465
Star 3
[ nan  nan  nan] [ nan  nan  nan]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [63]:
HD101274_1 = findmag(NGC3783_r,P_r,sta1_RA_WORLD,sta1_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_22_1 = findmag(NGC3783_r,P_r,sta2_RA_WORLD,sta2_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_529_1 = findmag(NGC3783_r,P_r,sta3_RA_WORLD,sta3_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)




NameError: name 'NGC3783_r' is not defined

In [ ]:
print 'r-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

In [ ]:
HD101274_1 = findmag(NGC3783_i,P_i,sta1_RA_WORLD,sta1_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_22_1 = findmag(NGC3783_i,P_i,sta2_RA_WORLD,sta2_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_529_1 = findmag(NGC3783_i,P_i,sta3_RA_WORLD,sta3_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)




In [ ]:
print 'i-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

In [ ]:
sta2_RA_WORLD = 174.82083333
sta2_DEC_WORLD = -37.76194444

HD101274_1 = findmag(NGC3783_z,P_z,sta1_RA_WORLD,sta1_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_22_1 = findmag(NGC3783_z,P_z,sta2_RA_WORLD,sta2_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_529_1 = findmag(NGC3783_z,P_z,sta3_RA_WORLD,sta3_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)




In [ ]:
print 'z-filter'
print HD101274_1
print np.nanmean(HD101274_1,), np.nanstd(HD101274_1)

print Tyc_22_1
print np.nanmean(Tyc_22_1), np.nanstd(Tyc_22_1)

print Tyc_529_1
print np.nanmean(Tyc_529_1), np.nanstd(Tyc_529_1)